In [3]:
P = 3221225473
F = GF(P)
F_gen = F.gen()
w = F.multiplicative_generator()
assert F_gen == 1

g = 1734477367 * F_gen
g_order = 8192

RS_factor = 8

gamma = g**RS_factor
gamma_order = sage.rings.integer.Integer(g_order / RS_factor)
trace_size = gamma_order

PR = F['x']

fib_1 = F_gen
last_fib_index = 1000
fib_last = F_gen * 2997353887

N_FRI_QUERIES = 2  # change this to two and find bug in mask paths

In [4]:
import hashlib

def enc_and_hash(data):
    return hashlib.sha256(str(data).encode("utf-8")).digest()

class FiatShamir():
    def __init__(self, seed=None):
        if seed is None:
            self.last_hash = b""
        else:
            self.lash_hash = seed

        self.nonce = 0
    
    def get_random_int(self, data):
        self.last_hash = hashlib.sha256(self.last_hash + enc_and_hash(data) + enc_and_hash(self.nonce)).digest()
        self.nonce += 1
        return int(self.last_hash.hex(), 16)

In [5]:
proof_example = ['eval_merkle:570761f588817960dc6d717177881d9be50ae3b58d8bf7d3b3d2a0be8571fbce', 'cp_eval_merkle:c0cb6706834f1e7e6e40ff92fd49b6888a5b300104516cf5238dc15ff37c8b99', 'layer_root:424c3e854bc258a7152a7d09ffc02b85926e7105394958b66fe534a9e04b4d19', 'layer_root:d582ff41551f3633d4f0b1169d997fbc33be8fdb60a76e0eb675dcb4ec19bd20', 'layer_root:b140030521cbbcfd9d3f1a20202c8628b5665381569f0818f0df8c6817f79836', 'layer_root:d7a19aa69905ad7f921deadd31f0122e809cad7cea9bc31f230aba959b2614b4', 'layer_root:1f03822c3eb8128415904cc7e1084291199f254d4a9364549cdd197a78a00570', 'layer_root:b883a4a69b051d80ae23fd9795343569827424f847a21675d3de11a6901534ac', 'layer_root:6bd48561e04f88c8755807e70de77987711fd4c0a64200fb12506baa8ac16d61', 'layer_root:09c12dc9937ba5cbf4770cd3cad52c2d142ed35efd4e94778b57fd0a79aa1ce3', 'layer_root:90460b3dbc7fba527fd94e804a680b289820409ea25d5e1772029c8af0ff1293', 'last_layer_const:1548814653', 'mask_eval:1664695642', "path:['3d6bded94d254eacef59ac852072c498798e7faa130bedf380a4d128f21b0f10', '3adc0d4e83d5c91afb558c80eb3c1c81b6057eb619c6e20fe2c83e5f92a5cf82', 'bc8d7ab2fa2c48bff720ef81d813d36fbe20081e6bfe1a3e34e67744681bf8f0', '4f6bc8b47921f602af13cd7965cc6cdc332df4513aa99f00aa03bf88d8eaed57', '75f6bf11532d106088a79da2cca0c750dd66263eddf899fd4b28779212f0f67f', '22f6a9c2f8e705fb21d0e8335d8fc16e54d6fc211cb5d37a9a2339b488440560', '068f7b9ea3d28cb975ae2c72b365ed23ff8ea9a06ff12198a408de3a80f04987', '36c7d147de12ca3e5785d9d6b27e8ba6d5af4aea771fb3c5fa8c39c0ea0f6312', 'ce5b033d3111099f62921caee42cd8e67ebb16c86048224152cc3142f9d5a7cd', '738e5fbc157457ebe5b03e9916f911ad540d65c5d229ecc2e7520a3538b87182', 'a06541af9265f3358d6eb0799a939259d24525f1e8f00af2032de931fec36587', '242e6e171073a6b2a02f9cc4dae85eca205c800f7f2e72977fc716f3a8e8bc4c', '0bd2e2cbd66617f60fdf312d7db7e01d107f6856b02b7f900a1e484625057e47']", 'mask_eval:1892597292', "path:['2a36bc678a956915c9785eec5706205e504eafb44428616666e91635e3b5cbbb', '06f413abb38faee312b7c8320cf539102e18fac4f195f7d0f126e1cf690087f6', '144cf5c869b4121ff5b93b4ab0b9e2df4f8d99250c289784ad7226f524b81c31', 'aab56c52fb982875326fd905973c65586d3ecf6b8ca292384a6473a0901ec977', '7be25aba452cf7515655249770c70254261f2c9446c43651d47584ded56f198a', '22f6a9c2f8e705fb21d0e8335d8fc16e54d6fc211cb5d37a9a2339b488440560', '068f7b9ea3d28cb975ae2c72b365ed23ff8ea9a06ff12198a408de3a80f04987', '36c7d147de12ca3e5785d9d6b27e8ba6d5af4aea771fb3c5fa8c39c0ea0f6312', 'ce5b033d3111099f62921caee42cd8e67ebb16c86048224152cc3142f9d5a7cd', '738e5fbc157457ebe5b03e9916f911ad540d65c5d229ecc2e7520a3538b87182', 'a06541af9265f3358d6eb0799a939259d24525f1e8f00af2032de931fec36587', '242e6e171073a6b2a02f9cc4dae85eca205c800f7f2e72977fc716f3a8e8bc4c', '0bd2e2cbd66617f60fdf312d7db7e01d107f6856b02b7f900a1e484625057e47']", 'mask_eval:311464662', "path:['dc187a49216689dbd0f37487833b8fbcc72ffc2ef2cc40dd6143cbbbe1da844b', '2c740db4d3d3f67a925600dde34be857e6952ae05b8324a1164edd3fd9ec7daf', '671503b9a261a19b42f18d5ace05a8790430f5bb1cbcdac0d92f4aee6e8d5400', '40395bd2bc687947793311154fbd32455e99403a64b3cdf1875bbea6cfa50085', '7be25aba452cf7515655249770c70254261f2c9446c43651d47584ded56f198a', '22f6a9c2f8e705fb21d0e8335d8fc16e54d6fc211cb5d37a9a2339b488440560', '068f7b9ea3d28cb975ae2c72b365ed23ff8ea9a06ff12198a408de3a80f04987', '36c7d147de12ca3e5785d9d6b27e8ba6d5af4aea771fb3c5fa8c39c0ea0f6312', 'ce5b033d3111099f62921caee42cd8e67ebb16c86048224152cc3142f9d5a7cd', '738e5fbc157457ebe5b03e9916f911ad540d65c5d229ecc2e7520a3538b87182', 'a06541af9265f3358d6eb0799a939259d24525f1e8f00af2032de931fec36587', '242e6e171073a6b2a02f9cc4dae85eca205c800f7f2e72977fc716f3a8e8bc4c', '0bd2e2cbd66617f60fdf312d7db7e01d107f6856b02b7f900a1e484625057e47']", 'layer_first_eval:2983035237', "path:['86bdcee778d0574343a3ae1488330575e4b0ad628936002566770e526a314bac', 'a0154089e8f84e17caf6406c8bd7152b15def014db1e489642e7b095347a39fb', 'abded45b5b40a0a6ad2091e926ee183c90a6ca50b2457eb3e674d432df75b92b', '5bfdd49a1cb451e75fab0be46121831b7519ce33aca0fa29aab8371659973f82', 'cf1cd201ab5db04a8675a9ea83892f0127941abd7ce4ed9ef1ab3a2c4fa3999c', '2408d7b569b528158f44c0fbb5ff1b7dde9be94fde1a49f1bc7f2983e56fee1d', '513c45e02b7eda4ca4b175463d63a49649e11dd2d6dfddd4dba67c90d62d8dda', 'c4868213c180227d578641d27fbe4520861016b0eb56c36937331af0709631e3', 'c332ab58ed3be7d21359a97afb005bd11d3c72251143d9135669af77ccb001f7', 'b6da6518ebd21b2cf815511f93831f2b75ca48665866c63e54e14686db9033d7', '98bdce98da0b86c7e67b8bf35472692ae8c77ed32f595cb6da9cba1af7c39ccc', '00f5957808ac79d5e83b0a7254f6523bbbab5f3bab38b41d7b9bcf13f3777e79', '1988225fc1015026648210c2a60ae6ce4e5b39930f8779108c1802f785fccb44']", 'layer_second_eval:2745862379', "path:['e7c147910e48a5a094b4f91afdcd8cb31f1c3fedb18c909007f4ddc1dc38e155', 'dc1a482cb21cd5e70ebe71fccae68d482b4356af0d30ecec4f5627bab0676b31', 'b0ade505c567db7ddaefb6b11e95930d66a7f7dd4e6eadff904abfde3adbb822', '127c2cad0a94e2beda6a8fe3bbf0b4658e6a513afcc7af99dc0fdc984e3eb200', 'f03dae0e2b4b430d3b4c50d2cb2c93dbd06b47e21a5debaca4a89b52b5431367', '4ba6ed467588981ef49428af4d4259db4be6f29f77154eea3490063bfa783dd4', '7da0ac45661aa239396e1d5be2d845ff59e6e055d4f41caa16a93b5982ccdc1b', 'fbaa9ae07866ac8556eca91d91df600921eec2e534f6d36bce53e09edef30535', '53624db830e0b703fb86cf1efcddb33efd5f713d5d2f8d44eaf281b56869ed3e', '7d483a0c667b980e2b10d6ba7a8503dac005bbe5ba0a070c181c65a50bf257d6', '5268d43ba84a5b6ce41469566ff096a3525c51ff8ad1706a304649014f622ab0', '229a721a796dfa684045464f01bf2df4acd8189f23812073184449d25a9ae2c0', '81bca2915c41812276fab3a55299303b1a6c9191d94375b15029ce8b5ca62802']", 'layer_first_eval:821264840', "path:['05741bdc9d728ccddf91136ce3103f9a00f588d3b4cd12f4743ca1b73d84f4e7', '36b5e52f754605d32766e602e064679b58911beb5543b696164ec93d80bc0088', '0ab5c61046ffa58d99474a891033c4f998c584f2c800802eff62d8993ff5b454', '1d5e229e5785d34d2139f43f1fc685e1b2f4383d208691f04df4bddb54503d6b', 'e32f5579eb042509a5cd1a9b8b399903c55ff01871c8162b654fa696d07fae92', 'e6e90b74bc1440462f8581db6660efbd3ac6ef40c867a82a9c058c0ff6472696', '6bb0efe85a355750273947ca9712b3c87f5fa2558b56d2dc300416b282d684aa', '9a145f183746c7b52c43f9bfc9c0ca696991b491a383b1a52f2238c85992c7d2', '754b04445353da27dfa02617da0061e6217cfb31e502f7aecbe50604fde53769', '97bc230a7a6801156f2d00a51fc70ecfbfd9a7bc9db910738854f9ecfef20ba9', '1292b53f30aa56f794369fb6fcf58b9f1e13e63556e512a1320fa0ae64893f79', '86960f31316cce27d665c8a9c789aa5095ae74dfc414040c1fd3670580466141']", 'layer_second_eval:1584526683', "path:['ba0ba41c9da63b752e68f63ed020afe550a2c085a4b5579b2b8d34e6bf6bcded', 'bfeabcef1c6de2bae51b2c35000b70cd1c26f17ec58248e8bb43b523307d64ec', 'f398c48a68d11df4a28dab2e50630e38de059718d1582c4f2dca4f53ade56b45', 'c006c103ad54d0efa7378b5eced61ef2ce13e1d754fc3a389c7c20960e0bc6fd', '6847996cba695e69b97bf2c3125d3078398c2fbba542697d647551d75db38d7f', '6cca4134b96bd6b5732c367e826a7e334959178d8991924de77a56143b938e13', 'c34ceeefc8b6b544ea1d7085d4441db200af9be18ebe3d9ae59451eebeef5ddc', 'd98efbf44a178e7e788103fb900c0590bdbba41b8d025edb2f4621e99ab5282f', 'a72d2fe858616a32bdce4054017eca41d51212eac28c6435e09671ac83dbed1e', '1afb0060598e042e9887ba704e9076be7a83252cd62ecb20d77e5b043e275a03', '12bd6e7c3ee31c1fe904210eb280e7774608d58cc1f11300807541122647cd83', '1267a62c86b1c85377ec8a0b63d0df6bb04edcaae06cdb0a4f3a9dc8c3761258']", 'layer_first_eval:215056732', "path:['4620573e7b92631509cc41448a795e2329fbf5f8305c85a5e026086d94b0f571', '257d6505c9f9baeeff77f9a6567107ee94cd2d3466791e098fd6379fdbc36182', '4edfe2d38ec78b528c529f2ad0fead1cba0ce69346da8fcda16b79b4a04069a7', '53ec53595d23666eec8b50482933b04210407fa1b93a065bedf63885f27f11c9', 'f7b544b26744f3019d4444e2ac9e53e3a403e59c9ef2ae62a4c67b70befc0579', '774ff7a22f133bf21318f7bcdc27e9587ac22aed8fde65a228da9d222aa31cbd', '83e24bd6279751d199322e4e330f43f3562cba9634c811b651c3c0bf2f9d792a', '4f7a3bf3c5510324c17f7808c95e829e065c1cbbfd68e73275c93fb56e2c79f3', 'e6dc5628b0446a7599da4666031ba0974ee1b65997fe3e2152fa68e5f733f514', '40612dd106c8b628189746111bfcec78fd17a2066804fa73cadc4b619aea3692', 'dde73226e0a6c5c0b1ad421216ca90aa748227408d814cfb83eaa34ca5e049b8']", 'layer_second_eval:375639587', "path:['1a9e5ccf860568a73439ffccd0b41dd4a63fb24276fc85f559123c570c665070', 'a7e3562ad0914507497e58d2218ca85d403669eac4f73ae2b791ada69ccc3f70', 'c98099887d685cbb99a939897439d718f9d3cdea02e97d3bc1e20968b79c0a61', '26684ab0cbb07823e826b497c1d02233a518cd50e50cb77c890e0d058dc9004a', '5b72bc3b075662a41ba51f57e4e100ae8ae00584fecb5c7843c7607b3600c2ec', '607fe48f7cce7ef780cae309acb93679ff2d84211d7d527bb7cf0a508960b0d0', '56e1a02936969e982735ecfe3cb847a6f5ce9698da9a7cce6c21b643bdc6a0e4', 'e6858f04423da52f9cc6865fc0fceccefbfa91376772c76cc58ce44a70ae2764', '3d73ab53b998b8c9b51f2b6944785ff3ec87e9d70819e239761eb3e2332fe579', '150f8cab808028b934e00a12dccee74b0c70d552cc2c974dde3debfbba6213ca', 'e2b882794a45de3dcbd9d385a3140c03f3c83bf34f6fc4605d7396b98a920ed2']", 'layer_first_eval:1858064921', "path:['41c25ab5cc3a09b4c3d60994bdc5b2e35ce60693ec941cc9cf351a41dfb23890', 'd30178a3a8f35323e3faa0322e9c1e45f7fe57eb73ab6e022fd68aef205bb29e', '994c1a5ce6d775652ca579a1442a04e931483894378576b5bf333d82401d4c70', '023912c94c4f0580fa554f50685fd181bcbea6f34485771fa3698ebcf968edb8', '8f07ae3b24e72f2a2a56f6e939561d0100f2849775b801a029516f77120b747c', '204b8663f45e0e82bc488cc494cd10a25a762c5753deda2dfb490a228fd2609b', '3b9d92f661dec851ad2290c6d079261abdfb77c01e31b14629ddaa34d535432d', 'f3744a127efa21704a31dd3ad817340c5f552067c10318ec6658945c4d8f814f', 'ddf4f5b9ddaa80dfc397ad764b39c027e9cd3a955375c9826025e57a008f0b44', 'fe34fc8de2db601a1c0835b129dee07b86423282ae207f984ea73070f7a18fad']", 'layer_second_eval:2439700484', "path:['029c7a762c9e1a925406dae27639a579debec80a658e7f779b1b62cef34f861f', 'bc7bdd4d9298c1555875d284fd70fce2602e7c312bfac03e633beb138edcbd52', 'f5a9ed82197e59b1c48ab9c899914574b5396ecacaf4ad9fe5c60b8743be9298', '2822a154e4a0b7c4a24121d7806b8be286fc1c7328f8fc7ece82fcd89d35677e', 'c659624756c619a9c2d4bac26d3f28ba15b00092277ed69f358ea89f19bc6b5a', '1562382084c89dc37eb262d5e365f176cecea3460e0c821c4efb72098fc87b55', 'e592b007cfe180a293d78cd60f00c63b1abb08bb9ae66579a0b903552e55f343', '74a45ab6775a9555c9280d1ea59894ddd7c67324b92ad03f9961a53a633e809c', 'ba56e0e63b421aa021fdb1ff4da6b4409981a6c202fae7d479ace159e265c757', '6ccd01bd37abc1f3f600c1cf4f92e07f44bce2b36fbeb07bd4455ef0f9d6bdb2']", 'layer_first_eval:3022965766', "path:['f69f91cb281d9897aa9b3917d3ec7d2af3180bdc9c2a8960f13aa95746b27426', '1b54b538af61c82f0f04f1850cf5ad170ffc2e9c03a861863e1ca470cfba92e9', 'd727dd0a019de3b0b36ea312942fca1250982b4ebbcb338c0ec730daff4791fd', 'e8265b7779155db8b08c40119e12949b41dca552fa7be15e9a1a3bc3981006b9', '9891efee7851f4f932f636f3e6e90d5471cab7841e81a28fb6e79660a540c6ed', '095a408e0d1d944e5708d5b78694400a445edeae579b10480fe44135d05e45a5', '365bc7afc1f13b67205f503eba0a2ed243570af33e01f7eda5934961bcfb738f', '628e7ff2460422477f675fb9cfe3562e09b7bbf35a5a7029e6204f87223b53fe', '3baf872e8b917b0cf1d4aa044c6c76366d86dafbc4fa0b6d52162d1e7a00b7ce']", 'layer_second_eval:3125144205', "path:['2eceb3126f5fcb55740078696a8e817296ed46f882accbdecf1da0f8c99c67e7', '261fefb07f7951cbbf31e1fd156f80568f5f4f9b009465972ce1d2ccb1eea5b2', '7b69818242cd9cb3e01ccd54637ecc6786e01e931b8526b7351a898a296790b0', 'f6af2fe07994b8be893dd165ddc672ed7589931057c1cf3f24023f41a67ec1be', '4c493435277734760da73431738a064ba0a5818972fcb8f63e78d3dd1c7acd1d', '64682a4ee6da964d89b98060ef2662e7d26b357c60e8d01e09893bd097cd9008', '762422740e53a3a5c227a37dcb3ef1e53fd3e37bed905e9189dd9cb73cbba6b5', '596467c66db658be812efb1947f6a18d640497ca3b71044ad59f494ab35a0716', 'b1b5890f3e8acc17184c5ed0538661cbabfc9a799e25ba09408c5efc1c21a975']", 'layer_first_eval:1734386712', "path:['cae451ef68083edd058de00a648ed660709087f40a78ac0b76bc8ed9d9652a0d', '87f9f9e0df43d51e52acad844ab9cab4e123df887bc8395e307f2778d9ad6e20', '344fff3518bbbd85d08c7f64f13c14ba7af0ee37098b5949a2222f66ee6a8d57', '4c523b545c6dfc3d7e026214ee0dd6e3c83f284198dac962e28ffb10b00d1d89', '7483a9af2945e0cc499ee6d9c97efd962039b457ff6a815da28cd19f643ba7b0', 'cb51abd560cd937ca8860b04e034446ade010345786c63ad2acd713558facec1', 'f5a6642af799b18a8cf23bfb772945b8a9fd32a3adc480297f9c00a5a49b6c79', 'afbe75239b93595f11bc3d5cfab01835be0b72aa6522bfe73736b54ef0e1b966']", 'layer_second_eval:2534634984', "path:['8d5daacd66fc8a30753c3295a85973185c00874e204d919e9f1c7891390a7d42', '45a435b43493ed5f0d86d8565251cdeff0b2b354287369efcecc5cfe318468fe', '3a011b853aa57ae2b4391719ed9b4adae8c3abf91b0a93136d0d292712456202', '5a827ed59433be18d307a50a21bd3d44fcdccfcb861f84707aef96f97a9e31b0', 'eb9841f0bee1b2902cca3413f37579171c27b38b96dc3ede548b061dc3156799', '3b67cc44d8ee9e87ed9ee145575411c6e5f2d05c6285b27bde1d44947fc7fcdc', '667ce71a18241def06806a5680f6f5cadd017fbc2420128541e1f1fee215e4dc', '939e4827ee84a82f75b1224279774ea2e1bb4e9222a07040bf17701ff7d4b8b3']", 'layer_first_eval:2145534578', "path:['402d8021df096f64a51ac36cf9b36bc32a2b07c096b02438011d14b9011e0c91', '7a9c2b8b97a43d454a69f5781c5e0bb1f3c32dd33f29c4f86d3a5ecd34df4b56', '8460e3c3bf03b65d4285010dc59db9ec3ef3a316cb1b19d930cabe556a983dda', 'da70b07c35ac95b72d73556ef422404b088b52511c5c64721070b0eaade495e4', '31531d56aa8488f1323cd100205837f515c74f25c27a8a6a8abfb2ee7e824f9f', '05c35f7b77f42eb2459d073fbcabaff0aed6dc559816c1dea8d09ca33ee356c1', '10e4c12f204554d93c6e913e2f43f8ebb93fadb374e779ad5b0f4dbe01870783']", 'layer_second_eval:1526398545', "path:['a9b7e43a24083cda5ef96818ee13f5fda0f76a59b89a24cd08cd2f2718172610', '51ac819b61ccd0ecbf8d2ede469c940ef6a0d2831ff090870406d56c2380dbea', '6ad350d656d2ca5f375bf55fb594107abef863ba618f852de81c6d02726af72e', 'c6fa79cf618ee5f96eaa3ab5d9b57e48908c88e3842e5dc74504d0e21444c8f5', 'b494d523abfcc394cb740ba79d360887234c9cba830741662266176788d6d1b2', 'f5cda635845652d96c08bb846060028e473637d6fe3dd35b785c21ea8f23dac2', '14450a0c6344f58bb19380f39e1076b982bf4c96e2310801679894330180b422']", 'layer_first_eval:1832231413', "path:['cf41ff6d204d4c534c0b567cfc75fd4a77ef45af5cb01620efd81fcde52bfb83', '35ca75f33f3a015a29a2954a14412b3cb55aacd2f971d72b14d03c2be6928216', '4264d8c17da1598536d5ea8d6b0e03fdb85ca7565ef9f55c1304c7677a44ed92', 'ab8bc55afbb8fca0a7b5f4c5aecf543b2347976c9b9d41d4f7580c5378a593a3', 'd2664f4dfbe78777727e7a02937ca9f7ba90d0963f8697dfc709e067450a5088', '83ba00d6b4feab098805c6951e32bc07cf9eafae824de4729f3ac04e2123f266']", 'layer_second_eval:1828000573', "path:['4d9b512288c497d869afdf866f224f7c1d5c81b70477b73ae1b2c4cf1b56c2d9', 'cc5f4751b07794a1b6608cd9c3242c6448f3160aaea306b537e46b6244c42688', '96afd103bca91bcfcd03f45e053e6d53e84abfc5e3df0b1586dfb19ae3cd063c', 'e078d06cdadbf5fa416ab65a2a1c665697cc50c4fbb0149ae6891ebafabc8e0c', '5dd7cd97efbd4a2e2f052af23518ff7b4d9a2f41cc7104a8f4f9c22eeecab7d4', '207bcefb3e12f16c93df25ca24d6409c40779d7274fbb4454165926389090bb4']", 'layer_first_eval:2945582939', "path:['afaddab5a86c503ce2842143b868a303b9935959854ee71eb6aed47e3d6bbf60', 'e1b888618df669f08c31a85fe15f4b593155cb5a247b3e787778419e620d98ac', '787279ecccd8321553efa9a96fa95bc1f4c219dc29a54bc640589ca6829c5d0a', 'e6340b977c95e0aca067edb7f2df3cb677cd4093e07a1de9fba0478b623f45ab', '2cf08a159e6858016baf00b83a6c985d27650eecbff7d0c6418e4643715572b6']", 'layer_second_eval:1916932023', "path:['86f1ff35377b3321086126d623d7d1737e39ccbfea61e2fb267692420989f41e', '511e05c41721e344d5c3429611687aab464280919896c7ca0e44eeb5a3182150', '45d7a7c00c4461c0d5ecd0a28d998709f47014cc70feb5201700df65798ff8a0', 'fd2eca6e52f5e6b750106b8c493bd6a7d586bf18bd8c04cea41597e3ba419990', '4eca91ea42ec5f888dc3af8e2622624ce278edf454355deff9369594abba9ef3']", 'layer_first_eval:2383446738', "path:['f07a70fa56a1ccd1bc8fa8b95df33a06132fc9c2f4e70e0409e354c3a12488ef', '517655eb608214ca130aa27f8637837c6be4ca6ecf2043458c7616eb68c515ce', '38752b7fb1abfb9c11b7437569430cd3b81d1ce23865c9bc9e229d614702b9e5', 'fb7cccceb65eb5a81530467cf90d0e5b4a4e5e5c4045fc89204ef0d87324b850']", 'layer_second_eval:1740588924', "path:['007e20d53c181be9f5f950886b8633b34cb057377c84831169969032d8185a40', 'ee822bf6e3fca6ecb85fce6a9e67f93ee8a9d50840a3568ad386dd035b17df8e', '02ecd1257919ca3a76ab9bb6feb0ad4dfbffdeea6a95b06db9f25c1df7b83936', '413e8f85fd7de2083f2cbf409395597b091905cc4ffc33622046a6c7e0b0d543']", 'mask_eval:1538950346', "path:['e68e79697adc78fac088a376cef5c33b2d70257abd6f923a846a9b0a42c802b6', '3783d1e71432d4032ecaedccb13c2e26308b7fe33ac1c7d6bf0af36ddd78cd23', '4d0f8483efec9f60191a0cc95e25d62765a94e302cd875d8d03fb077230ab137', 'd6d9e2982088045b7ee6a46de7354259d9f86cde9d9885501eca240a4cf9faa9', 'fd05a73b6304ae7ea4b8107c1871e775fa3878b01173a8b23e122918f253d001', '0ed43e4f3a57b0db7de5d0c597e3037c352858e76640a5e7833d76e18cdd85f9', '79e6a22eafdc3c668573b7e35966505147ddc60043c56e85092e8cb56a60f1b0', '484bd9e55c98026c269430c8ea15f235cc13d87731523bb7deff62da3803feaf', '189f2fcdc0c727fbe639d5d07ef9147c75a12ca904d73ee92f981b594a185ed7', 'a170e23f1561ad5137d5c57d3afc00ad8bf0d91b53c895dc07e1644df8435451', 'edc66683fa0e069c6f88c9e00ef8e2105458b5648c9c8b711abbddeea53d08bd', 'ed47ff0c8ce61c50430b42e6f5b2028becc1dca21c9c3f14ea92f860cae6a39b', '0bd2e2cbd66617f60fdf312d7db7e01d107f6856b02b7f900a1e484625057e47']", 'mask_eval:2260423425', "path:['5f76e9b72e430d21faf38f6bb786e3fe58702f4b68f3ebd1222c38ae26e9550f', 'b975211c50e0655842e11157b7937827c4b0dc291eb240420dfe2a9460b7c977', 'e2a074229f0bd6fb029a38f1056bed37b18ab8e821b47b587ec581cdfcc2b805', '8c130c93672f3e9c2f3b61869cb6a6116fb401704523fbe810b75597c9eadc97', 'b8e583954d02ebcf1dd54f52a1b0248613a15d60d6ec15704301cdd0c257d245', '887d3aacbbd5e9ad2aebd6fbe6b913457a5f8158c6c30245a3cf72f6e5569360', '538de1006bca47e7a68b53ab59d6dbc6eb348705e117051adc7f7c7e2b568173', 'bd12468c06db866052cb59715bb2c88963b6fecc7cd24b3609ef9a6a216ff990', '189f2fcdc0c727fbe639d5d07ef9147c75a12ca904d73ee92f981b594a185ed7', 'a170e23f1561ad5137d5c57d3afc00ad8bf0d91b53c895dc07e1644df8435451', 'edc66683fa0e069c6f88c9e00ef8e2105458b5648c9c8b711abbddeea53d08bd', 'ed47ff0c8ce61c50430b42e6f5b2028becc1dca21c9c3f14ea92f860cae6a39b', '0bd2e2cbd66617f60fdf312d7db7e01d107f6856b02b7f900a1e484625057e47']", 'mask_eval:2768477723', "path:['1236f720efba2c0ff5d61db9a7cb8c22250d6feeb6a223614379d5190174d726', '1497b5fd62961572e5c926f4cbd261b6e41c333595e92cb208c7592dd6912e3f', '960e8240f976de8bdd92a8be419f4c4d7e0c4084f7fad44dc25dca18c938908e', '1a5ea08dfce011f4440f115cf7e2735add63c081546e9c85093bd188a69100c1', 'b8e583954d02ebcf1dd54f52a1b0248613a15d60d6ec15704301cdd0c257d245', '887d3aacbbd5e9ad2aebd6fbe6b913457a5f8158c6c30245a3cf72f6e5569360', '538de1006bca47e7a68b53ab59d6dbc6eb348705e117051adc7f7c7e2b568173', 'bd12468c06db866052cb59715bb2c88963b6fecc7cd24b3609ef9a6a216ff990', '189f2fcdc0c727fbe639d5d07ef9147c75a12ca904d73ee92f981b594a185ed7', 'a170e23f1561ad5137d5c57d3afc00ad8bf0d91b53c895dc07e1644df8435451', 'edc66683fa0e069c6f88c9e00ef8e2105458b5648c9c8b711abbddeea53d08bd', 'ed47ff0c8ce61c50430b42e6f5b2028becc1dca21c9c3f14ea92f860cae6a39b', '0bd2e2cbd66617f60fdf312d7db7e01d107f6856b02b7f900a1e484625057e47']", 'layer_first_eval:2303836870', "path:['d37104b771119107f3ec945203823e3405525536817e50fd362f96407ef9beec', 'eaa971147b20935c8f4452944269001b134a369c3d8a1db0db8e5f3065ad4b94', 'aab6f8a61235b9d8da6262eadb59b3e9ad1d7985fd29b2cbf24881e60af2e99f', 'c4a294f77a977aa785d6ba3868819f11b92fa19d728ef87d20fc3115647e5c62', '86edd938f8d37e8535f76bb351fdef347a6ecffbfa6fb607be0f3a8e7cc6fe59', 'b2b329de8a2e66e0e1068266b953a2e70a7abb0bf6cbfbe784a81a67efed8cf0', '89be35f463de14136195c3516a56325e4dc923e20584fd0dbb457dcfc865bbd3', 'df8c4765f9b5611ee12bf2a62540e948c148a162b8219b9415afb89b0cc20967', 'ea44ff8f87783813e9fae5e2e416ed2a8989dd5c572686056e3aa8cadf1da4b2', 'f31c89e44d4faf35b20a13348adba7c2d64739f975b118fda09ee4d361bca89b', 'f0ffee9693ef3f0b16bfea0a287b7a39d6c34bf037d032e3305e819d1788da9c', '44ea327632693cdb709c5d72617d59895ee1355fab22712e2d277a3719dfa7fd', '1988225fc1015026648210c2a60ae6ce4e5b39930f8779108c1802f785fccb44']", 'layer_second_eval:3115878777', "path:['d9692e7d08cff85a4923d2515591a9c825c313d14d6bf353ec4e01d2d0ffdf5a', '802f7e953930277bfd7f962c1a9d67ba50b7b2fee21fee42fb535c747fd1168f', 'd1aebafe67f1ef186c674485e9278002ec42c34a018e435775542e321218031d', 'be7c8c4026a65ae86c2409baf463abb393b591666d1aef24e71b4d9433d10bc4', '4e0773aea0f2a593dc815d312303c331530f112ce2120f1c19e5ab50658c3e0a', '1b42cc50623fe78dafd6b5b00f010075e48bce38ac5d605d73107be8032bfca9', '474da2cc46df1c1927f7b267cbf6fee127e3a10cd1b31a9ee720b5de4168e914', '8f6dcd3d150a96662520edc7c96be805b493d8ce5dcdb53fbd76bd5984cd73c8', '2c1622fcb60df18054492ddac71eded7328c62d29e81c71a6fb6cd63354bfd4c', 'e24d0e6d175a0c88eb2242f17fdebbbeb4e8683c94739641f320c20b73cd75ec', 'e0d3e5e03314c02b23660207567ff36869f7a343a7d0d514a6ba9dfefc691680', 'f6ff1ff0b49254bc7364a82a6c0bb319c70e082f05495d89c1c073758f49cc90', '81bca2915c41812276fab3a55299303b1a6c9191d94375b15029ce8b5ca62802']", 'layer_first_eval:2870145221', "path:['23bc983fa62db0d734f04a0cc92393517ae479e5bf87754adefb5e9ab9189f43', '8c3dcd20060dde16dc2eceff11489aabbec5e816eb001582c15003033d7309b6', '062bfc7639dc6cf08690afea39c8c50971b1d6fa9bcd3e76749e55c5f83ea602', '3699b851b3e52793ff0a725645218e77c5c4bcf59a71c1f1673a03e80ea11422', 'efaf60b185740b88621a97bbbc408ef593e156b852e55d390500ee70ccdef9c3', 'e575edddeb4ec2d64436dd9e7d9c3334b05bec6816d1de9d6007d2b3560d6c18', 'a96f41d6ea63274ce5bfed4ab85cdbdd28ce1639214eb4efd2b14179f0cd5e5a', '0c862db876683f76e7231dff78511359e91a6171844763f9686b6bf11f8b7ce1', 'beeaa682fd531cd19d6b4df68e87be29d977627fe05606514ddc17413044300c', 'd30407ae923db71db03381c864a427464ba6e3f7f2639ac98c895983414fcbc2', '12bd6e7c3ee31c1fe904210eb280e7774608d58cc1f11300807541122647cd83', '1267a62c86b1c85377ec8a0b63d0df6bb04edcaae06cdb0a4f3a9dc8c3761258']", 'layer_second_eval:897369152', "path:['74430d65b6522f90e0d831f45c78a6d3364d6b12d7d1eb4e58cfee0e85eb1cec', 'e0ac1cb824d75791a98137a8782e5113dc5ac6af108916221c532073a3cf2094', '7725e39e425e68984d1516ccbdf844c73fed1951167fcc4c7543a0776da44cbc', 'a09094f7b5f527b4390f6d16569dbf47eb7d3c30af79edd9ec78c81034995897', 'c560402fd4fb3d097683d292f9bb7c8a1438f0b720dfc469f9b7c8a542e51b2a', '9f3af56bcf579852a69cbeff2582a3332861c20e87df15f54f1c508f42357f9b', '0d36d73f6bfba121f1bbf8e09662b20a52f553f39bbd8c5722dc155fb9d1a202', 'c652a86b944dbec76c11fde3717a3e55c6e1a9d81fc14bc04d4975134ca3d7ce', '28c3a325249d13ef62a27cc1592d33aecc52eb71a8ebc30b9d95a6b9ba0f9b96', 'b483fc3ca0f161c3840ac058367006207bec264e5b703d8c255c6871e81f41c0', '1292b53f30aa56f794369fb6fcf58b9f1e13e63556e512a1320fa0ae64893f79', '86960f31316cce27d665c8a9c789aa5095ae74dfc414040c1fd3670580466141']", 'layer_first_eval:710769993', "path:['98184bd56ca4b99a3b04d09422c9bd984ae150417a6cdfa58f939529f0244f51', 'c547e7fd0555dca2c688d90d8e4e1c812c2e01ee465cff1871a9c7b19c265c24', '8b964af474bed5bbf9340af0932d06844f2d6a0f7274995416d023701785ace5', '484ed81273849389da5d8deb69628dca2585a3474414ed859ddf5c89f45c4009', '139c654a3f1014c1e3221de4d66eba2527d7ef849872eb6cbd256aad7a77f6d8', '692e788a9d4f5c88941df2c011334b509b4277e5981aa1c903cab3c1f57bd7fd', 'fb89149a77963e23d3a4618a9d6812e33cc59d815759785cfd73272b001756e1', 'ca49e7b80bb3de5470cb329d73c7cc03ade4fd768dcbcb872f2cdbab20c33664', '70549f1575f23a9394a314ce6bf1c96e68139233c05cb45d6ab909bb6ff5b46c', '0cae36543a4335382eb67a317a457d5cdc427981622ccd3e2bdc82a9571977a4', 'dde73226e0a6c5c0b1ad421216ca90aa748227408d814cfb83eaa34ca5e049b8']", 'layer_second_eval:2956918269', "path:['f959daebe0408dce06aa1ed11e6a68fab4eef37279904b47b07f9bb6fd2db9ad', 'ae6f657aaaa3237ecc421f1858a9861d623e1af51a2439ce7fb619281f1b55ba', '76a6e98c9464f078e4a13139b3c3ae17317a3e64f2e85c59b57a17b5cc20dd91', '5fe3668c499b677023f5a7916e4981464592a2e64b3744e10757f7f0fd4ce5cf', '3a8a64c49dc27f970838b484e5df738d61c850a2fb1349432e5dad7fcc960fcf', 'e39a0f318c74e21e3f8a97d0a494b972e6595a822487dc8b28b9a1906e5d5ab8', '58c0d15ddf494386f96bae578db0f4b98c12859870a51f73ab3149bec2386aa5', '25cd1957a7a52e8c7a97d1b7ada6d34c5325fc1595103c785c080716f03e4b77', 'b33e68f8e5d7d73f79930a59cb76bac7cfcc0a9b00e5623c4422a23474970cef', '68b65fa699a80a2614fd7bb9d3f2a42160f9ecd805504c1d3cd65cc96502115c', 'e2b882794a45de3dcbd9d385a3140c03f3c83bf34f6fc4605d7396b98a920ed2']", 'layer_first_eval:1561033738', "path:['ac6f5e3b198cdd9721d8141746711e2aadbee7ea1459f6bc742e5273311b1e65', '1a91e73111dd241e8a341916a52efc3e352fc698cc25b475c8b9fe0bb09328e5', '357b85e9b22efe78e97aff47999838c6b5116e1af0a2b364b768ddca55cc80f9', 'f8491ce316fc797346b1ae9e2aa42cbf23840f14fb924b36e3d0bb5645451255', '3ae0e29fa58f5f461b4106c7061c39d16774ac0a219dc4cc323871d39e4fcf2a', 'aa6710959d629534ff97ea0d739a8a8c1f175391978e52fbc91f1c74a3ef9d5d', '0412a8e229dbbe0ce2069499dbdb8f0c4a0e952d0fba747025a29a6200ea199f', '74a45ab6775a9555c9280d1ea59894ddd7c67324b92ad03f9961a53a633e809c', 'ba56e0e63b421aa021fdb1ff4da6b4409981a6c202fae7d479ace159e265c757', '6ccd01bd37abc1f3f600c1cf4f92e07f44bce2b36fbeb07bd4455ef0f9d6bdb2']", 'layer_second_eval:2287197310', "path:['68464903d3f7f6e426525c6c8deb17b0644ea3f517325ef22d2dacd96b9f3c46', 'f5bc883813cee50023480e346f48501dbbc6b399ff16d5d5956c16e11473a055', '9f2e34018f62943d9f7c890c14632fdc377b0363158e52e8f7c04373512e269d', 'd8d6f6c311934325cf08ea46f1334bda71fd525afffed20dd8bb0e7a588b0c87', 'f59c79a7d6bd432fcc16c78c36dab8369996e66d5841d325214166f86441bc58', '6c1b594969b6c56ea15c88b1bcf89c3d31a2ebfbf5046f69e72d530e74bc3605', '06a408b91caab19e396e79ea2f553dfb7891933059db85a1c8cbdd790afe2bd8', 'f3744a127efa21704a31dd3ad817340c5f552067c10318ec6658945c4d8f814f', 'ddf4f5b9ddaa80dfc397ad764b39c027e9cd3a955375c9826025e57a008f0b44', 'fe34fc8de2db601a1c0835b129dee07b86423282ae207f984ea73070f7a18fad']", 'layer_first_eval:781671195', "path:['0ba492e8d2739405f4839a44cb653e1e22cc5785b8cb89c98498f89934b88452', 'b90e71c59b82c4c0312301f0bf98cbd8b9e9c7a00d71fb1a52ab51fdda5bb9f5', '8ca53bb43123d3b012cc1ff694f2685124eb4dcf4d8a04e7b4ac5db13fbb1941', '6262865fab67f1193fb94c9ee17119b412c02d0beb1213e07edfe806a880c80d', 'f96f7b7861552e2231dde95d40ed56ddcf9147b1c55468bf5d71289757759770', 'f6c500e2996b75e1b005ac71a1719e7fe3b439facdc854540f046a9b7e833b66', 'd96fba4e856847054edf05f239d74bb468a7c0ef0100890043543c889352d71e', '628e7ff2460422477f675fb9cfe3562e09b7bbf35a5a7029e6204f87223b53fe', '3baf872e8b917b0cf1d4aa044c6c76366d86dafbc4fa0b6d52162d1e7a00b7ce']", 'layer_second_eval:490285763', "path:['66453df971e8790cf1a88e81cdba7cdbdb82a76b7c947e44b574257d1fc63f85', '70012dad0e86d3c8a3c9a75c46d2ceeca3ba4fccd0aefb26e3b752f19b38e639', '8ea1d51019cca9e03383957d099dec87f1f4ec0172f6ad3d8e2b66434f4cf69f', '8b66d85353f71b728e7c9fd1a2668b359f5c13ec3211ee396b2c81a37f4dc82f', 'c3b0918c017e9b0594f433b7621044b12260793e2a5bb9cfba6498f289cf8031', 'a44a82f465c779c95ed1694c32ce7dcae96ff0e27992c7683cc732d2e0d2f7d0', '45ff312c8ea84aed3a735acfaa79067b2dadcf2dc79add7eedfbef95fb0c36df', '596467c66db658be812efb1947f6a18d640497ca3b71044ad59f494ab35a0716', 'b1b5890f3e8acc17184c5ed0538661cbabfc9a799e25ba09408c5efc1c21a975']", 'layer_first_eval:954771307', "path:['10d3cca4be56ff7b1e977fb5d1b4dd43c5ebc4eaf27d03ae06c28a3b255b7b2c', 'f8acee4fde5b926fb2ef722f2fd1381718e65a447bf2d2066172f42d8d119bd8', '00dca1a156f604fb4f1453e2bea51f9652f5ace5f3d7e0fbed74b4a90ddcc231', '0a4628dc0d57dbf993c55267dee89a8049977f468a8993979bb4c0aa22822c95', 'a8406ef5d2fa2b04cc83b6875b1d893811500776c93771ebb9c013ee07d6e90d', '5d6f03ad334a48a9f3aba6c02f00dac5a1392a13c0946e37d344350aaf7e4ced', '73b4cc01ffeb44bfc9a5e2718646b5e912b9eb86ab57642db7cbad8c88e24cab', 'afbe75239b93595f11bc3d5cfab01835be0b72aa6522bfe73736b54ef0e1b966']", 'layer_second_eval:2574382659', "path:['76d94b6f57effd2fe7321995a53ed23d5ff5bf32b46749e7c762dc1cc764d045', '5ddbc5ab94259130568fe2a8c3e4007beec9627ff68547565d3e4f76244657c3', '729a9d57dc9035c8ad57a71c3d6dfb5f6cb07b1cdafc6fbfd35561a612be85a9', '1d4130bf12e1c23c9e717fb96b5f4cc22a47bfc82a44a013125359df67fb4082', 'fac5208769c1c125de5d71e8d28e41138c6a125242397264c819bb986438fff9', '5b834ef42fd4499247e8031f9a77cc774d56109c103c0198b67d7b6e7e6d3616', 'd307d8b6d01296a177067c51835c8d4da6083bdeeefe9ad0d50b7b7e60129769', '939e4827ee84a82f75b1224279774ea2e1bb4e9222a07040bf17701ff7d4b8b3']", 'layer_first_eval:127750681', "path:['edb22f35ef62c00a7a4e7c80cafbfa1e6ca196d38795024c59fdc682457d5d19', '0af902229fe565c4ce76e7f7d58405892759983bc86a220b7f2bc2f2f4bbb3d6', 'dba9977e9d04f19ad81e293a455132851c7e867debddf8001822109aa301cb6f', '0e0b8878cfd816d4bfb1c72a3d7005abc7b2687176428296790a3f00fdfe4f8e', 'b127edfcdcb44f6e9ccbbcac503ac4f6ef9d77caf51614a8ae47d56fb55ec6be', 'f5cda635845652d96c08bb846060028e473637d6fe3dd35b785c21ea8f23dac2', '14450a0c6344f58bb19380f39e1076b982bf4c96e2310801679894330180b422']", 'layer_second_eval:838434126', "path:['d67d51944958ab8b020e3dc59b40afd1c4ba66da5ec7816249c67864a3745052', '8fa1ef53ed850cdf05f739b36a65ee2055a204f46dbcdaedf7b828ae8b47ce4d', 'fdc40c0463f545b6b03d91891d8ed27f52f73fc5cfeaf8cedbba1da3b247bfa7', '0d8e9d27193b7351e992234b146a01f5370867d4799c54d2b6f2740dca2fbf1a', '2de8a26dd54cc0e8cabe83f61ce72fe00248280aac805b9355f618bf173df9b8', '05c35f7b77f42eb2459d073fbcabaff0aed6dc559816c1dea8d09ca33ee356c1', '10e4c12f204554d93c6e913e2f43f8ebb93fadb374e779ad5b0f4dbe01870783']", 'layer_first_eval:412060029', "path:['ae643250626999d173426e8a697bc257de7ebe8cec1c065d0fcf1bfb142727a0', '27880fc1868d81d56047a1465ed06e441e71cdca03c08ef41ec3458ee416f923', '615d7c3d723e2119d7da4a1847c5f08ae343e75917627ff704ee9be045a0c31c', '8865f9139668bdeabd01cf1b5da0008237477a15f503b6927f181c42fb4029c2', '86287e3480ee8a2448c34e9e51dd195513ce9c175a8e14d608100d43b8c3755d', '83ba00d6b4feab098805c6951e32bc07cf9eafae824de4729f3ac04e2123f266']", 'layer_second_eval:340096725', "path:['a8bc42b12da03594229d712179a1c8d0a53a902a1078b06f76d24c5f7c72b059', 'e00b8501fb9396bb02ae5b9829012c5a3cc2c50652e1d55bc27d977178c7240c', '7ba7970ebcdacd48bf0160484b95b8dd4feaf1c55905073b3b2edcae15c2fe41', '96549ca20d7f385086fb0b4fdc7bf20fcb0f8d51d38eb25c39c9a35a4f2fc327', '17b013a47f6153c91a2ff940610787c0033d01911ccff280c5bfb84f991a049b', '207bcefb3e12f16c93df25ca24d6409c40779d7274fbb4454165926389090bb4']", 'layer_first_eval:2908154745', "path:['c37822d63e312c0b021b2b0d52603af57b90384e2af8484137513808a9490b89', 'a982b52a41d46d83345291e9c4defa7f7a0694ef5593ebbd7618e593f3cf4eac', '45d7a7c00c4461c0d5ecd0a28d998709f47014cc70feb5201700df65798ff8a0', 'fd2eca6e52f5e6b750106b8c493bd6a7d586bf18bd8c04cea41597e3ba419990', '4eca91ea42ec5f888dc3af8e2622624ce278edf454355deff9369594abba9ef3']", 'layer_second_eval:2401629173', "path:['b7bdc703d27cbe5bfbe8b751aa1fc336a0c84bcffcb7c0fbb7eadf1c26ab659c', '61c1fe5f014c57defea0fa8055c424670c7e2090e9001bdf39710774ef17f5bb', '787279ecccd8321553efa9a96fa95bc1f4c219dc29a54bc640589ca6829c5d0a', 'e6340b977c95e0aca067edb7f2df3cb677cd4093e07a1de9fba0478b623f45ab', '2cf08a159e6858016baf00b83a6c985d27650eecbff7d0c6418e4643715572b6']", 'layer_first_eval:702005785', "path:['176ed47ef3b56b67d7843978071dcf351b6fdabedc324bb04564d90e11ba7228', 'b0c90dce826c37e21ce99fa47176d39ec590909b25bb9d09c6cbc220d1ce208b', '38752b7fb1abfb9c11b7437569430cd3b81d1ce23865c9bc9e229d614702b9e5', 'fb7cccceb65eb5a81530467cf90d0e5b4a4e5e5c4045fc89204ef0d87324b850']", 'layer_second_eval:200804404', "path:['cec5a8d3e06b77f97ff182800e49ce4ad0530f443405002b9e8488bdf3fa1a1d', '343059a242011142bb09c1515fbaf641cbf1fa3c9e5e16ce815f8f98d8a8176f', '02ecd1257919ca3a76ab9bb6feb0ad4dfbffdeea6a95b06db9f25c1df7b83936', '413e8f85fd7de2083f2cbf409395597b091905cc4ffc33622046a6c7e0b0d543']"]